# Pydantic basics
* work with data in an OOP manner
* fields and methods
* validate data
* serialize - OOP object -> json
* deserialize - json -> OOP object

### start without pydantic
how to naively create validation for a Person class

- error handling
- property
   - getter
   - setter (put in validation here)

In [1]:
class Person: 
    def __init__(self, name, gender, age):
        self.name = name
        self.gender = gender
        self.age = age


p1 = Person("Gauss", "M", 73)

# default __repr__ for the Person class
p1
        

In [2]:
class Person:
    def __init__(self, name , gender, age):
        self.name = name
        self.gender = gender
        self.age = age

    def __repr__(self):
        return f"Person(name='{self.name}', gender='{self.gender}', age={self.age})"


p1 = Person("Gauss", "M", 73)

p1

Person(name='Gauss', gender='M', age=73)

In [3]:
p2 = Person('Dadda', 'F', 3)
p2

Person(name='Dadda', gender='F', age=3)

In [4]:
p1.age , p1.name 

(73, 'Gauss')

In [5]:
p2.age , p2.name 

(3, 'Dadda')

create a totally invalid person

In [6]:
# this should give error
Person(3.141592, 2.715, 'TwentyFive')

Person(name='3.141592', gender='2.715', age=TwentyFive)

some other programming languages have static typing or hard typing

in hard typing language

```

int age;
string name;
string gender;

```

it will give error when giving wrong data types to the variable

```
age = "femtiotvå"; // -> gives an error
```


in python we have dynamically typed language lets try type hinting

In [7]:
class Person:
    def __init__(self, name: str, gender: str, age: int):
        self.name = name
        self.gender = gender
        self.age = age

    def __repr__(self):
        return f"Person(name='{self.name}', gender='{self.gender}', age={self.age})"

Person(3.14, "M", 53)

Person(name='3.14', gender='M', age=53)

### add property

In [8]:
class Person:
    def __init__(self, name: str, gender: str, age: int):
        self.name = name
        self.gender = gender
        self.age = age

    @property
    def age(self):
        print("getter called")
        return self._age
    
    @age.setter
    def age(self, age):
        print("setter called, validate age code here")
        self._age = age

    def __repr__(self):
        return f"Person(name='{self.name}', gender='{self.gender}', age={self.age})"
    
p3 = Person('henry', gender = 'M', age = 32)
p3

setter called, validate age code here
getter called


Person(name='henry', gender='M', age=32)

In [9]:
p3.age 

getter called


32

### validation on property

In [ ]:
class Person:
    def __init__(self, name: str, gender: str, age: int):
        self.name = name
        self.gender = gender
        self.age = age

    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age: int):

        if not isinstance(age, int):
            raise TypeError(f"age must be an int, not type {type(age)}") 

        if not 0<=age<=125:
            raise ValueError(f"age must be between 0 and 125, not {age}") 
        
        self._age = age

    def __repr__(self):
        return f"Person(name='{self.name}', gender='{self.gender}', age={self.age})"

try:
    p4 = Person("henry", gender = "M", age = "fem")
except TypeError as err:
    print(err)

try:
    Person("henry", gender = "M", age = -539128)
except ValueError as err:
    print(err)

age must be an int, not type <class 'str'>
age must be between 0 and 125, not -539128


## Pydantic for same example

In [12]:
from pydantic import BaseModel, ValidationError


class Person(BaseModel):
    name: str
    gender: str
    age: int

# -2 for age is okay here, but that's not good
p5 = Person(name="Babba", gender="F", age=-2)
p5

Person(name='Babba', gender='F', age=-2)

In [13]:
try:
    Person(name=1.41, gender=78, age="två")
except ValidationError as err:
    print(err)

3 validation errors for Person
name
  Input should be a valid string [type=string_type, input_value=1.41, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=78, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='två', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


In [14]:
from pydantic import Field
from typing import Literal

class Person(BaseModel):
    name: str 
    gender: Literal["M", "F"] = Field(description="gender, valid values are 'M' and 'F'")
    age: int = Field(0, gt=-1, lt=126, description="age of a human person")

try:
    Person(name="Diddi", gender="m", age=126)
except ValidationError as err:
    print(err)

2 validation errors for Person
gender
  Input should be 'M' or 'F' [type=literal_error, input_value='m', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be less than 126 [type=less_than, input_value=126, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than


### serialize pydantic model

make it into json

In [15]:
p6 = Person(name="Doddo", gender="F", age=5)
p6

Person(name='Doddo', gender='F', age=5)

In [16]:
type(p6.model_dump())


dict

In [19]:
# dict
p6.model_dump()

{'name': 'Doddo', 'gender': 'F', 'age': 5}

In [20]:
# json string
p6.model_dump_json()

'{"name":"Doddo","gender":"F","age":5}'

In [21]:
print(p6.model_dump_json(indent=3))


{
   "name": "Doddo",
   "gender": "F",
   "age": 5
}


In [22]:
with open("person.json", "w") as file:
    file.write(p6.model_dump_json())

### deserialization

* go from json to pydantic model

In [23]:
with open("person.json", "r") as file:
    person_data = file.read()

p9 = Person.model_validate_json(person_data)
p9

Person(name='Doddo', gender='F', age=5)

In [24]:
p9.name, p9.age

('Doddo', 5)